## Collaborative Filtering

A collaborative filtering algorithm usually works by searching a large group of people and finding a smaller set with tastes similar to ours. It looks at other things they like and combine them to create a ranked list of suggestions.

In [1]:
'''
We shall use a self-prepared nested dictionary as a small exemplary dataset.
'''
import recommendations
from recommendations import critics
# exemplary run
# check_1---------------------------------------------------
critics['Lisa Rose']['Lady in the Water']

# check_2---------------------------------------------------
critics['Toby']['Snakes on a Plane']=4.5
critics['Toby']

{'Snakes on a Plane': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 1.0}

## Finding Similar Users
This can be achieved by comparing each person with every other person and calculating a *similarity score*.
There are a few ways to achieve this score. Two of them which I shall be using here are:
1. Euclidean Distance
2. Pearson Correlation

### Euclidean Distance Score

In [2]:
from math import sqrt

# Returns a distance-based similarity score for person1 and person2
def sim_distance(prefs, person1, person2):
    # Get the list of shared_items
    si = {}
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item]=1
            
    # if they have no ratings in common, return 0
    if len(si)==0: return 0
    
    #Add up the squares of all the differences
    sum_of_squares = sum([pow(prefs[person1][item]-prefs[person2][item], 2)\
                          for item in prefs[person1] if item in prefs[person2]])
    
    return 1/(1+sum_of_squares)

In [3]:
# exemplary run
round(sim_distance(recommendations.critics, 'Lisa Rose', 'Gene Seymour'), 6)

0.148148

In [4]:
'''
Lets us find out the similarity index of each person against the rest.
'''
def find_similarity(scorer):
    all_critics = []
    for p1 in recommendations.critics.keys():
        all_critics.append(p1)

    done_matches = []
    print("Similarity score:")
    for p1 in all_critics:
        for p2 in all_critics:
            compare = [p1, p2]
            compare.sort()
            if p2 is not p1 and compare not in done_matches:
                print("{} and {}: {}".format(p1, p2,
                                             round(scorer(recommendations.critics, p1, p2), 6)
                                            )
                     )
                done_matches.append(compare)
            

In [5]:
'''
Find similarity score for all
'''
find_similarity(sim_distance)

Similarity score:
Jack Matthews and Mick LaSalle: 0.137931
Jack Matthews and Claudia Puig: 0.181818
Jack Matthews and Lisa Rose: 0.210526
Jack Matthews and Toby: 0.117647
Jack Matthews and Gene Seymour: 0.8
Jack Matthews and Michael Phillips: 0.181818
Mick LaSalle and Claudia Puig: 0.173913
Mick LaSalle and Lisa Rose: 0.333333
Mick LaSalle and Toby: 0.307692
Mick LaSalle and Gene Seymour: 0.129032
Mick LaSalle and Michael Phillips: 0.285714
Claudia Puig and Lisa Rose: 0.285714
Claudia Puig and Toby: 0.235294
Claudia Puig and Gene Seymour: 0.133333
Claudia Puig and Michael Phillips: 0.571429
Lisa Rose and Toby: 0.222222
Lisa Rose and Gene Seymour: 0.148148
Lisa Rose and Michael Phillips: 0.444444
Toby and Gene Seymour: 0.108108
Toby and Michael Phillips: 0.285714
Gene Seymour and Michael Phillips: 0.210526


### Pearson Correlation Score
**Algoithm:**
1. First find the items rated by both critics.
2. Calculate the sums and sum of squares of the ratings for the two critics
3. Calculate the sum of products of their ratings.
4. Use the results to calculate Pearson Correlation coefficient.

In [6]:
# Returns the Pearson correlation coefficient for p1 and p2
def sim_pearson(prefs,p1,p2):
    # Get the list of mutually rated items
    si={}
    for item in prefs[p1]:
        if item in prefs[p2]: si[item]=1
    
    # Find the number of elements
    n=len(si)
    
    # if they are no ratings in common, return 0
    if n==0: return 0
    
    # Add up all the preferences
    sum1=sum([prefs[p1][it] for it in si])
    sum2=sum([prefs[p2][it] for it in si])
    
    # Sum up the squares
    sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
    sum2Sq=sum([pow(prefs[p2][it],2) for it in si])
    
    # Sum up the products
    pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
    
    # Calculate Pearson score
    num=pSum-(sum1*sum2/n)
    den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    if den==0: return 0
    r=num/den
    return r

In [7]:
# Exemplary run
print sim_pearson(recommendations.critics,'Lisa Rose','Gene Seymour')

0.396059017191


In [8]:
'''
Find similarity score for all
'''
find_similarity(sim_pearson)

Similarity score:
Jack Matthews and Mick LaSalle: 0.211289
Jack Matthews and Claudia Puig: 0.028571
Jack Matthews and Lisa Rose: 0.747018
Jack Matthews and Toby: 0.662849
Jack Matthews and Gene Seymour: 0.963796
Jack Matthews and Michael Phillips: 0.13484
Mick LaSalle and Claudia Puig: 0.566947
Mick LaSalle and Lisa Rose: 0.594089
Mick LaSalle and Toby: 0.924473
Mick LaSalle and Gene Seymour: 0.411765
Mick LaSalle and Michael Phillips: -0.258199
Claudia Puig and Lisa Rose: 0.566947
Claudia Puig and Toby: 0.893405
Claudia Puig and Gene Seymour: 0.31497
Claudia Puig and Michael Phillips: 1.0
Lisa Rose and Toby: 0.991241
Lisa Rose and Gene Seymour: 0.396059
Lisa Rose and Michael Phillips: 0.40452
Toby and Gene Seymour: 0.381246
Toby and Michael Phillips: -1.0
Gene Seymour and Michael Phillips: 0.204598


## Ranking the Critics

In [9]:
# Returns the best matches for person from the prefs dictionary
# Number of results and similarity function are optional params.
def topMatches(prefs, person, n=5, similarity=sim_pearson):
    scores = [(similarity(prefs, person, other), other)
              for other in prefs if other!=person]
    
    # Sort the list so the highest scores appear at the top
    scores.sort()
    scores.reverse()
    return scores[0:n]    

In [10]:
# Calling function with a name gives us a list of movie critics
topMatches(recommendations.critics,'Toby',n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

## Recommending Items
- We score the items by producing a weighted score that ranks the critics.
- Take the votes of all the other critics and multiply how similar they are to me by the score they gave each movie.

In [11]:
# Get recommendation for a person by using a weighted average
# of every other user's rankings
def getRecommendations(prefs, person, similarity=sim_pearson):
    totals = {}
    simSums = {}
    for other in prefs:
        # don't compare me to myself
        if other==person:
            continue
        sim=similarity(prefs,person,other)
    
    # ignore scores of zero or lower
        if sim<=0:
            continue
        for item in prefs[other]:
        
        # only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item]==0:
                # Similarity * score
                totals.setdefault(item, 0)
                totals[item] += prefs[other][item]*sim
                # sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+=sim
            
    # Create the normalized list
    rankings = [(total/simSums[item],item) for item,total in totals.items( )]
    
    # Return the sorted list
    rankings.sort()
    rankings.reverse()
    return rankings

In [12]:
# Exemplary run 1
getRecommendations(recommendations.critics,'Toby')

[(3.3477895267131013, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]

In [13]:
# Exemplary run 2
getRecommendations(recommendations.critics,'Toby', similarity=sim_distance)

[(3.5002478401415877, 'The Night Listener'),
 (2.7561242939959363, 'Lady in the Water'),
 (2.461988486074374, 'Just My Luck')]

## Matching Products
- We can determine similarity by looking at who liked a particular item and seeing the other things they liked.

In [16]:
def transformPrefs(prefs):
    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, {})
            
            # Flip item and person
            result[item][person] = prefs[person][item]
    return result

In [17]:
movies = transformPrefs(recommendations.critics)
topMatches(movies, 'Superman Returns')

[(0.6579516949597695, 'You, Me and Dupree'),
 (0.4879500364742689, 'Lady in the Water'),
 (0.11180339887498941, 'Snakes on a Plane'),
 (-0.1798471947990544, 'The Night Listener'),
 (-0.42289003161103106, 'Just My Luck')]

The above results show that those who like Superman Returns tend to like movies The Night Listener and Just My Luck.

In [19]:
getRecommendations(movies, 'Just My Luck')

[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]

In [20]:
# TODO: represent the correlation on a graph